# Python 调用 C++

In [1]:
from utils import set_vta
import numpy as np
import torch
from torch import nn
import tvm
from tvm.ir.module import IRModule
# from tvm.script import tir as T
from tvm import te, topi
from utils.dll import load_dll
load_dll("./libs/libtvm_ext.so")

<CDLL './libs/libtvm_ext.so', handle 564e8b302db0 at 0x7f2ac3550d00>

In [2]:
myadd = tvm.get_global_func("myadd")

In [3]:
myadd(22, 23)

args: 22 23

45

In [4]:
@tvm.register_func("tvm.contrib.add")
def add(x, y, z):
    print(f"x:\n{x}\ny:\n{y}")
    tvm.nd.array(x.asnumpy() + y.asnumpy()).copyto(z)

In [5]:
n = 10
A = te.placeholder((10,), name="A")
B = te.placeholder((10,), name="B")
C = te.extern(
    A.shape,
    [A, B],
    lambda ins, outs: tvm.tir.call_packed("tvm.contrib.add", ins[0], ins[1], outs[0]),
    name="C",
)

In [6]:
sch = te.create_schedule(C.op)
te_func = tvm.lower(sch, [A, B, C])
# te_func = te.create_prim_func([A, B])
# te_func.show()
f = tvm.build(te_func, "llvm")
a_np = np.random.uniform(size=(n,)).astype(A.dtype)
b_np = np.random.uniform(size=(n,)).astype(B.dtype)
c_np = a_np + b_np
a = tvm.nd.array(a_np)
b = tvm.nd.array(b_np)
c = tvm.nd.array(np.random.uniform(size=(n,)).astype(C.dtype))
f(a, b, c)
np.testing.assert_allclose(c.asnumpy(), c_np, rtol=1e-5)

x:
[0.444      0.8509563  0.18271057 0.26373902 0.3103753  0.9174619
 0.20557065 0.37674654 0.1447992  0.2188932 ]
y:
[0.39630783 0.57694376 0.84100753 0.00503178 0.03568824 0.60567266
 0.09110498 0.76846087 0.7763797  0.24185385]
